In [328]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')

In [329]:
url = "http://localhost:8056"

In [330]:
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

In [331]:
import random

In [332]:
new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", name="test")



DEBUG: Making POST request to http://localhost:8056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '628927@example.com', 'name': 'test'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Thu, 17 Apr 2025 11:03:54 GMT, Thu, 17 Apr 2025 11:03:53 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '111', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"628927@example.com","name":"test","image_url":null,"id":74,"created_at":"2025-04-17T11:03:54.192833"}...


In [333]:

user_id = new_user['id']

In [334]:
user_id

74

In [335]:
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']


DEBUG: Making POST request to http://localhost:8056/admin/users/74/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Thu, 17 Apr 2025 11:03:54 GMT, Thu, 17 Apr 2025 11:03:53 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '115', 'content-type': 'application/json'}
DEBUG: Response content: {"user_id":74,"id":76,"token":"9JsOBRwmvXKr6HRvbB81Uax7BAGsBzopCj9fQogo","created_at":"2025-04-17T11:03:54.224363"}...


In [336]:
user_api_key

'9JsOBRwmvXKr6HRvbB81Uax7BAGsBzopCj9fQogo'

In [337]:
client = VexaClient(
    base_url=url,
    api_key=user_api_key
)

In [338]:
meeting_url="https://meet.google.com/qcm-yvzc-izx"
native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"

In [339]:
meeting_info = client.request_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    bot_name="MyBot"
)
meeting_id = meeting_info['id']


DEBUG: Making POST request to http://localhost:8056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': '9JsOBRwmvXKr6HRvbB81Uax7BAGsBzopCj9fQogo'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'qcm-yvzc-izx', 'bot_name': 'MyBot'}


DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Thu, 17 Apr 2025 11:03:54 GMT, Thu, 17 Apr 2025 11:03:53 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '394', 'content-type': 'application/json'}
DEBUG: Response content: {"id":101,"user_id":74,"platform":"google_meet","native_meeting_id":"qcm-yvzc-izx","constructed_meeting_url":"https://meet.google.com/qcm-yvzc-izx","status":"active","bot_container_id":"99decb692a305b5302876e7814d59e6606b1c119012385430cb61d8d3b6e726a","start_time":"2025-04-17T11:03:54.495853","end_time":null,"created_at":"2025-04-17T11:03:54.262304","updated_at":"2025-04-17T11:03:54.267841"}...


In [340]:
meetings = client.get_meetings()


DEBUG: Making GET request to http://localhost:8056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': '9JsOBRwmvXKr6HRvbB81Uax7BAGsBzopCj9fQogo'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Thu, 17 Apr 2025 11:03:54 GMT, Thu, 17 Apr 2025 11:03:54 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '409', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":101,"user_id":74,"platform":"google_meet","native_meeting_id":"qcm-yvzc-izx","constructed_meeting_url":"https://meet.google.com/qcm-yvzc-izx","status":"active","bot_container_id":"99decb692a305b5302876e7814d59e6606b1c119012385430cb61d8d3b6e726a","start_time":"2025-04-17T11:03:54.495853","end_time":null,"created_at":"2025-04-17T11:03:54.262304","updated_at":"2025-04-17T11:03:54.267841"}]}...


In [341]:
meeting_id

101

In [342]:
import pandas as pd

In [343]:
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [375]:


transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
df.tail(10)


DEBUG: Making GET request to http://localhost:8056/transcripts/google_meet/qcm-yvzc-izx
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': '9JsOBRwmvXKr6HRvbB81Uax7BAGsBzopCj9fQogo'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Thu, 17 Apr 2025 11:05:11 GMT, Thu, 17 Apr 2025 11:05:11 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1794', 'content-type': 'application/json'}
DEBUG: Response content: {"id":101,"platform":"google_meet","native_meeting_id":"qcm-yvzc-izx","constructed_meeting_url":"https://meet.google.com/qcm-yvzc-izx","status":"active","start_time":"2025-04-17T11:03:54.495853","end_time":null,"segments":[{"start":0.0,"end":3.0,"text":" Четыре, пять, раз, два, три, четыре, пять.","language":"en","created_at":null,"speaker":null},{"start":4.0,"end":5.0,"text":" Ну-ка, ну-ка, ну-ка, ну-ка, ну-ка, ну-ка, ну-ка.","language":"en","created_at":null,"speaker":null},{"start":4.1,"end":...


,start,end,text,language,created_at,speaker
2,4.10,6.10,"Ну-ка, ну-ка, ну-ка, ну-ка, ну-ка.",en,None,None
3,6.10,8.10,"Пап, смотри. Не надо.",en,None,None
4,8.10,10.10,"5-6-7, 5-6-7.",en,None,None
5,10.10,14.63,"А ну-ка, ну-ка, ну-ка,",en,None,None
6,14.63,16.63,"ну-ка, ну-ка, ну-ка.",en,None,None
7,16.63,18.63,12 копченых гусей,en,None,None
8,18.63,20.63,"пришли, поругали",en,None,None
9,20.63,27.29,поругали и присели. 12 копченых поросят.,en,None,None
10,31.21,40.15,И чтобы это не значило.,en,None,None
11,31.71,40.15,И чтобы это не значило.,en,None,None


In [376]:
df

,start,end,text,language,created_at,speaker
0,0.00,3.00,"Четыре, пять, раз, два, три, четыре, пять.",en,None,None
1,4.00,5.00,"Ну-ка, ну-ка, ну-ка, ну-ка, ну-ка, ну-ка, ну-ка.",en,None,None
2,4.10,6.10,"Ну-ка, ну-ка, ну-ка, ну-ка, ну-ка.",en,None,None
3,6.10,8.10,"Пап, смотри. Не надо.",en,None,None
4,8.10,10.10,"5-6-7, 5-6-7.",en,None,None
5,10.10,14.63,"А ну-ка, ну-ка, ну-ка,",en,None,None
6,14.63,16.63,"ну-ка, ну-ка, ну-ка.",en,None,None
7,16.63,18.63,12 копченых гусей,en,None,None
8,18.63,20.63,"пришли, поругали",en,None,None
9,20.63,27.29,поругали и присели. 12 копченых поросят.,en,None,None
